# Reaching multiple targets with a manipulator
The objective of this exercice is to introduce the implementation of the front-end based on Pinocchio, for polyarticulated systems modeled in position-velocity-torque.

We provide a basic example for reaching one point with a manipulator robot. You are then guided to modify this example for sequence of multiple targets. 

## Front-end and back-end
Our optimal control toolbox is composed of 3 main parts: the backend contains the API for solving the problem and accessing the solution and implements several solvers, DDP being the first one; the frontend is first composed of a basic API which we mostly used in the exercice with unicycle and bicopter, and that you can use to implement any fancy problem; finally, a particular implementation of the front end using Pinocchio is mostly written for working with polyarticulated systems such as manipulator robots, 



## Set up

In [ ]:
import gepetuto.magic

# Panda reaches a single target

## Preparing the environment 
We will need crocoddyl as in the previous notebook, with the model of the arm of the humanoid robot Talos, a 7-dof arm. It can be found in example robot data.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_import
import crocoddyl
import pinocchio as pin
import numpy as np
import example_robot_data as robex
import matplotlib.pylab as plt
import time


First, let's load the Pinocchio model for the Panda arm.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_robexload
# First, let's load the Pinocchio model for the Panda arm.
robot = robex.load('panda')
# The 2 last joints are for the fingers, not important in arm motion, freeze them
robot.model,[robot.visual_model,robot.collision_model] = \
    pin.buildReducedModel(robot.model,[robot.visual_model,robot.collision_model],[8,9],robot.q0)
robot.q0 = robot.q0[:7].copy()


The optimal control problem is defined by a bunch of hyperparameters.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_hyperparameters
HORIZON_LENGTH = 100
TIME_STEP = 1e-2
FRAME_TIP = robot.model.getFrameId("panda_hand_tcp")
GOAL_POSITION = np.array([.2,0.5,.5])
GOAL_PLACEMENT = pin.SE3(pin.utils.rpyToMatrix(-np.pi,0,np.pi/4), GOAL_POSITION)
REACH_DIMENSION = "3d" # "6d"


Set robot model.

In [ ]:
# %load tp5/generated/arm_example_robot_model
# Set robot model
robot_model = robot.model
robot_model.armature = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.])*5
robot_model.q0 = np.array([3.5,2,2,0,0,0,0])
robot_model.x0 = np.concatenate([robot_model.q0, np.zeros(robot_model.nv)])
robot_model.gravity *= 0


We will use meshcat for displaying the model.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_viz
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
viz.addBox('world/goal',[.1,.1,.1],[0,1,0,1])
viz.applyConfiguration('world/goal',GOAL_PLACEMENT)


In [ ]:
viz.viewer.jupyter_cell()

The model is augmented with a default state $x_0$, and armature is added to the joints to model the gear reduction.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_robot_model
# Set robot model
robot_model = robot.model
robot_model.armature = np.ones(robot.model.nv)*2 # Arbitrary value representing the true armature
robot_model.q0 = robot.q0.copy()
robot_model.x0 = np.concatenate([robot_model.q0, np.zeros(robot_model.nv)])


## Build the action model combining cost and dynamics
This starts with specifying the state space, defined by $x=(q,v)$. 

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_state
state = crocoddyl.StateMultibody(robot_model)


The cost is defined as a sum of multiple terms. The classes CostSum are created to store the terms. Let's have one for the running nodes, and one for the terminal node.

We need to first define a cost model (i.e. set of cost functions) in order to next define the action model for our optimal control problem.
For this particular example, we formulate three running-cost functions: goal-tracking cost, state and control regularization; and a terminal cost: goal cost. First, let's create the common cost functions.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_sumofcosts
runningCostModel = crocoddyl.CostModelSum(state)
terminalCostModel = crocoddyl.CostModelSum(state)


We define a cost for reaching the target, either as a 3D objective $\ell(x) = \| p(q)-p^*\|^2$, or as a 6D objective $\ell(x) = \| \log( \ ^0M_E(q)^{-1} \ ^0M_* ) \|^2$, with $p^* \in \mathbb{R^3}$ the goal position, and $^0M_* \in SE(3)$ the goal placement.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_cost_goal
if REACH_DIMENSION == "3d":
    # Cost for 3d tracking || p(q) - pref ||**2
    goalTrackingRes = crocoddyl.ResidualModelFrameTranslation(state,FRAME_TIP,GOAL_POSITION)
    goalTrackingCost = crocoddyl.CostModelResidual(state,goalTrackingRes)
    runningCostModel.addCost("gripperPose", goalTrackingCost, .001)
    terminalCostModel.addCost("gripperPose", goalTrackingCost, 1)
elif REACH_DIMENSION == "6d":
    # Cost for 6d tracking  || log( M(q)^-1 Mref ) ||**2
    goal6TrackingRes = crocoddyl.ResidualModelFramePlacement(state,FRAME_TIP,GOAL_PLACEMENT)
    goal6TrackingCost = crocoddyl.CostModelResidual(state,goal6TrackingRes)
    runningCostModel.addCost("gripperPose", goal6TrackingCost, .001)
    terminalCostModel.addCost("gripperPose", goal6TrackingCost, 1)
else:
    assert( REACH_DIMENSION=="3d" or REACH_DIMENSION=="6d" )


Next we add regularization. First a state regularization.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_cost_xreg
# Cost for state regularization || x - x* ||**2
# We set up different values for the integral cost and terminal cost term.

# Regularization is stronger on position than velocity (to account for typical unit scale)
xRegWeights = crocoddyl.ActivationModelWeightedQuad(np.array([1,1,1,1,1,1,1, .1,.1,.1,.1,.1,.1,.1]))
xRegRes = crocoddyl.ResidualModelState(state,robot_model.x0)
xRegCost = crocoddyl.CostModelResidual(state,xRegWeights,xRegRes)
runningCostModel.addCost("xReg", xRegCost, 1e-3)

# Terminal cost for state regularization || x - x* ||**2
# Require more strictly a small velocity at task end (but we don't car for the position)
xRegWeightsT=crocoddyl.ActivationModelWeightedQuad(np.array([.5,.5,.5,.5,.5,.5,.5,  5.,5.,5.,5.,5.,5.,5.]))
xRegResT = crocoddyl.ResidualModelState(state,robot_model.x0)
xRegCostT = crocoddyl.CostModelResidual(state,xRegWeightsT,xRegResT)
terminalCostModel.addCost("xReg", xRegCostT, .01)


Then a control regularization.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_cost_ureg
# Cost for control regularization || u - g(q) ||**2
uRegRes = crocoddyl.ResidualModelControlGrav(state)
uRegCost = crocoddyl.CostModelResidual(state,uRegRes)
runningCostModel.addCost("uReg", uRegCost, 1e-6)


## Action model 
Next, we need to create an action model for running and terminal nodes. We follow the same logic already explained with the bicopter: a differential action model (DAM) for the forward dynamics and the cost integrals, then a numerical integration in the integrator action model (IAM).

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_iam
# Next, we need to create the running and terminal action model.
# The forward dynamics (computed using ABA) are implemented
# inside DifferentialActionModelFullyActuated.

# The actuation model is here trivial: tau_q = u.
actuationModel = crocoddyl.ActuationModelFull(state)
# Running model composing the costs, the differential equations of motion and the integrator.
runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModel), TIME_STEP)
runningModel.differential.armature = robot_model.armature
# Terminal model following the same logic, although the integration is here trivial.
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModel), 0.)
terminalModel.differential.armature = robot_model.armature


## Optimal control problem 
Once we have the action models, we just have to shape them into an optimal control problem.


In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_shoot
problem = crocoddyl.ShootingProblem(robot_model.x0, [runningModel] * HORIZON_LENGTH, terminalModel)


## Solve and display
We finalize the set up by creating the DDP solver for this optimal control problem.

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_solve
# Solving it using DDP
# Create the DDP solver for this OC problem, verbose traces, with a logger
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([
    crocoddyl.CallbackLogger(),
    crocoddyl.CallbackVerbose(),
])

# Solving it with the DDP algorithm
ddp.solve([],[],1000)  # xs_init,us_init,maxiter


You can now look at the results, either by plotting it or animating the trajectory in the viewer.

In [ ]:
%matplotlib inline

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_plot
# Plotting the solution and the DDP convergence
log = ddp.getCallbacks()[0]
crocoddyl.plotOCSolution(log.xs, log.us, figIndex=1, show=False)
crocoddyl.plotConvergence(
    log.costs,
    log.pregs,
    log.dregs,
    log.grads,
    log.stops,
    log.steps,
    figIndex=2,
    show=False,
)


In [ ]:
viz.viewer.jupyter_cell()

In [ ]:
# %load tp2/generated/panda_reaches_a_single_target_animate
# # Visualizing the solution in gepetto-viewer
for x in ddp.xs:
    viz.display(x[:robot.model.nq])
    time.sleep(TIME_STEP)

# Extending the example for reaching a sequence of targets

Now we ask you to modify this example to reach a sequence of targets $p_{*1},p_{*2},p_{*3},p_{*4}$. The optimal trajectory should use similar regularization, but with the reaching cost now varying it time: for the first quarter of nodes, the target is $p_{*1}$, then another quarter with $p_{*2}$ etc until the four targets are reached. Don't specify a particular velocity when reaching the point to let more freedom to the solver.

Below is a quick guideline to help you.

### First step: prepare the environment

Start by defining several targets (let's say 4 targets, all at x=0.4, and at y and z being either 0 or 0.4), and display then in the viewer.


### Second step: define the shooting problem

The shooting problem will be composed of 4 sequences of action models. Each sequence consists on T shooting "running" nodes and 1 terminal node. The running nodes mostly have regularization terms, while the terminal nodes have a strong cost toward the respective target.

$[ R_1,R_1,R_1 ... R_1,T_1, R_2,R_2 .... R_2, T_2, R_3 ... R_3, T_3, R_4 ... R_4 ] , T4

First create 4 running models and 4 terminal models.

Then you need to add a position cost, and state and control regularization to each running action model. Please  note that for terminal action model is only needed the position cost. Additionally, in the running models, the position cost should be low, and it should be high in the terminal models.

Now create a shooting problem.

In [ ]:
seq0 = [runningModels[0]]*T + [terminalModels[0]]
seq1 = [runningModels[1]]*T + [terminalModels[1]]
seq2 = [runningModels[2]]*T + [terminalModels[2]]
seq3 = [runningModels[3]]*T 
problem = crocoddyl.ShootingProblem(x0,seq0+seq1+seq2+seq3,terminalmodel[3])

Create a DDP solver for this problem and run it. 

In [ ]:
ddp = crocoddyl.SolverDDP(problem)
ddp.solve()

Well, it should not work, at least no on the first shot. The DDP solver is likely not strong enough to accept the random weights that you have selected. 

If it is working nicely from the first shot, display it in the viewer and go take a coffee. But you will likely have to tweak the gains to make it work.

**It is suggested to first optimize only sequence 1. When you are happy with it, add sequence 2 and optimize again, etc.**


### Opening: toward hard constraints

The solver works with double precisions, so it is quite robust to high weight. 10000 is likely to be accepted for example. But if you make the problem too difficult, the solver will break. 
In that case, you can implement a simple penalty solver by setting the weight to be 10**i, and creating a for loop to explore i from 0 to 5. At each iteration of the loop, run the solver from the previous solution and for few iterations only.

for i in range(1,6):
    for m in terminalModels:
        m.costs.costs['gripperPose'].weight = 10**i
    ddp.solve(ddp.xs, ddp.us, 10)

This not very convenient, and a better solver should be used if you really want imposing hard constraints. Let's do that next.

----
# Adding hard constraints

In this last chapter, we guide you through the advanced, more recent, constraint interface. It is composed of two parts. First, constraints can be defined in the front end, following a similar logic to constraints, and using again the residual models. That should be pretty straight forward now you are proficient with cost models. Second, we need another solver. DDP is not aware of constraints and would just skip them. More advanced solvers are available. We propose here to use the recent SQP from the team "Machine in Motion" led by Ludovic Righetti at NYU, described in TODO ARXIV URL. TODO URL to MIM

The implementation  is available in the mim-solver package.

In [ ]:
import mim_solvers

## Defining the constraints

Similarly to the sum-of-costs, all constraints must be stored in a constraint manager, which is given as initial argument when building a differential action model.

We propose here to introduce a virtual wall constraining the end effector. For that, we use the same frame-translation residual. The constraint is defined along each axis with constant bounds, and np.inf when a particular direction should not be considered.

Then you can build a new action model with this constraint. We advise you don't constrain the initial node, as it easily leads to a unfeasible problem if $x_0$ does not satisfy the constraints. 

## Working with the SQP solver of MIM
The solver follows a very similar syntax to the DDP solver. 

## Do it yourself.
Now take the 4-point OCP you wrote in the first part. The motion finally obtained is typically a circle containing the four target points. 
### First exercice
Add some bounds to sharpen the circle, so that the arcs are flatten by the virtual walls.
### Second exercice
Additionally add some constraints for the targets points, rather than defining them as costs. The constraints should then be equality, while they where inequalities up to now. This can be implemented by puting two equal bounds.
### Third exercice
Add joint limits. You will use the residual state for that. The bounds can be obtained from the URDF model through:

In [ ]:
robot_model.lowerPositionLimit, robot_model.upperPositionLimit, robot_model.velocityLimit